Ce script fait deux choses : 
1. Il génère automatiquement un fichier CSV fictif (donnees_test_livraison.csv) qui simule votre cas d'usage (prédiction de retard de livraison).
2. Il lance ensuite PyCaret sur ce fichier.

Ce que vous allez voir :
1. Le script va créer le fichier à côté de votre script Python.
2. Il va nettoyer les données (transformer "Camion/Avion" en chiffres automatiquement).
3. Il va afficher un tableau de classement avec le Score F1, l'Accuracy, etc.
4. La variable best_model contiendra l'algorithme gagnant prêt à être utilisé.

In [1]:
import pandas as pd
import numpy as np
from pycaret.classification import *

# --- ÉTAPE 1 : CRÉATION DU FICHIER DE DONNÉES (Simulation) ---
# On génère 500 lignes de fausses données clients/livraisons
np.random.seed(42)
n_samples = 500

data = {
    'ID_Client': range(1, n_samples + 1),
    'Distance_km': np.random.randint(5, 500, n_samples),
    'Poids_Colis_kg': np.random.uniform(0.5, 30, n_samples).round(2),
    'Type_Transport': np.random.choice(['Camion', 'Avion', 'Train'], n_samples),
    'Prioritaire': np.random.choice(['Oui', 'Non'], n_samples),
    'Prix_Commande': np.random.randint(20, 1000, n_samples),
    # La colonne cible (1 = Retard, 0 = A l'heure)
    'Retard_Livraison': np.random.choice([0, 1], n_samples, p=[0.7, 0.3]) 
}

# Création du DataFrame et sauvegarde en CSV
df_fictif = pd.DataFrame(data)
nom_fichier = 'donnees_test_livraison.csv'
df_fictif.to_csv(nom_fichier, index=False)

print(f"✅ Fichier '{nom_fichier}' généré avec succès !")
print("-" * 50)

# --- ÉTAPE 2 : VOTRE CODE AUTOML (PyCaret) ---

# 1. Chargez vos données
print("Chargement des données...")
data_reelle = pd.read_csv(nom_fichier)

# 2. Initialisez l'environnement
# On ignore 'ID_Client' car ce n'est pas une info prédictive
# La cible est 'Retard_Livraison'
print("Initialisation de PyCaret (Le 'Heavy Lifting')...")
s = setup(data=data_reelle, 
          target='Retard_Livraison', 
          ignore_features=['ID_Client'], 
          session_id=123,
          verbose=False) # verbose=False évite trop de texte à l'écran

# 3. Comparez tous les modèles
print("Comparaison des modèles en cours (patience...)...")
best_model = compare_models()

# 4. Affichage du meilleur modèle
print("-" * 50)
print("Le meilleur modèle trouvé est :")
print(best_model)

✅ Fichier 'donnees_test_livraison.csv' généré avec succès !
--------------------------------------------------
Chargement des données...
Initialisation de PyCaret (Le 'Heavy Lifting')...
Comparaison des modèles en cours (patience...)...


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7000,0.4996,0.0000,0.0000,0.0000,0.0000,0.0000,0.9430
ridge,Ridge Classifier,0.7000,0.5005,0.0000,0.0000,0.0000,0.0000,0.0000,0.0280
lda,Linear Discriminant Analysis,0.7000,0.5005,0.0000,0.0000,0.0000,0.0000,0.0000,0.0350
dummy,Dummy Classifier,0.7000,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0310
nb,Naive Bayes,0.6857,0.5168,0.0000,0.0000,0.0000,-0.0244,-0.0276,0.0370
ada,Ada Boost Classifier,0.6571,0.5296,0.2000,0.3670,0.2486,0.0605,0.0653,0.1430
catboost,CatBoost Classifier,0.6571,0.4910,0.1245,0.2912,0.1683,0.0104,0.0059,1.2630
rf,Random Forest Classifier,0.6514,0.4837,0.1918,0.3273,0.2350,0.0414,0.0402,0.1410
knn,K Neighbors Classifier,0.6457,0.5317,0.1709,0.2927,0.2131,0.0198,0.0135,1.0890
xgboost,Extreme Gradient Boosting,0.6429,0.5406,0.2773,0.3952,0.3115,0.0855,0.0935,0.4470


--------------------------------------------------
Le meilleur modèle trouvé est :
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=123, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


Le but de cet algorithme est très simple : transformer votre historique de données en une "boule de cristal" fiable.

En termes plus techniques, c'est de l'Apprentissage Supervisé (Supervised Learning).

Voici une explication imagée pour comprendre ce qui se passe sous le capot :

1. L'Analogie du "Concours de Recrutement"
Imaginez que vous êtes le directeur d'une entreprise de livraison. Vous avez un énorme livre de bord (votre fichier CSV) qui contient l'historique de 10 000 livraisons passées. Pour chaque livraison, vous savez si elle a été En retard ou À l'heure.

Vous voulez embaucher un expert capable de deviner si la prochaine livraison sera en retard, juste en regardant le bon de commande.

Le code que vous lancez (compare_models) fait ceci :

Il convoque 15 experts différents (ce sont les algorithmes : CatBoost, Random Forest, XGBoost, etc.).

Il leur donne vos données passées pour qu'ils "révisent". Chaque expert essaie de comprendre la logique cachée (ex: "Ah, quand c'est un camion et qu'il pleut, c'est souvent en retard").

Il leur fait passer un examen : Il cache les résultats d'une partie des données et demande aux experts de deviner.

Il vous désigne le vainqueur : Celui qui s'est trompé le moins souvent.

2. Concrètement, avec vos données :

Les Indices (Inputs) : Le poids du colis, la distance, le type de transport (Avion/Camion).

La Cible (Output) : La colonne Retard_Livraison (0 ou 1).

Le but du jeu : L'algorithme cherche des motifs invisibles à l'œil nu.

Peut-être qu'il découvrira que les colis de plus de 20kg transportés par "Train" sur une distance de >300km sont en retard 90% du temps.

Une fois qu'il a appris cela, vous pouvez lui donner un nouveau dossier (une livraison qui n'a pas encore eu lieu) et il vous dira : "Attention, risque de retard détecté !".

En résumé
Ce code remplace l'intuition humaine ("Je sens que ça va être en retard") par une probabilité mathématique calculée sur des faits réels.

Maintenant je vous montre comment utiliser le modèle gagnant pour prédire le retard sur de nouveaux clients fictifs ! :D

C'est parti ! Voici la suite logique du code.

Imaginez que nous sommes le lendemain matin. Vous avez deux nouvelles commandes sur votre bureau et vous voulez savoir si vous devez vous inquiéter.

In [2]:
# --- ÉTAPE 3 : UTILISER LA BOULE DE CRISTAL (PRÉDICTION) ---

# 1. Créons de nouveaux profils de livraison (ceux de "demain")
# Note : On doit avoir les mêmes colonnes que pour l'entraînement
nouveaux_dossiers = pd.DataFrame({
    'Distance_km': [800, 15],          # Cas 1 : Loin / Cas 2 : Près
    'Poids_Colis_kg': [25.5, 1.2],     # Cas 1 : Lourd / Cas 2 : Léger
    'Type_Transport': ['Camion', 'Avion'],
    'Prioritaire': ['Non', 'Oui'],
    'Prix_Commande': [150, 1200]
    # Pas de colonne 'Retard_Livraison' ici, c'est ce qu'on cherche !
})

print("Analyse des nouveaux dossiers en cours...")

# 2. Le modèle fait sa prédiction
predictions = predict_model(best_model, data=nouveaux_dossiers)

# 3. Affichons les résultats de façon lisible
# PyCaret ajoute deux colonnes à la fin : 
# - 'prediction_label' (La réponse : 0 ou 1)
# - 'prediction_score' (La certitude : de 0 à 1)

print("-" * 50)
print("RÉSULTATS DE LA PRÉDICTION :")
print("-" * 50)

# On renomme pour que ce soit plus clair à lire
resultat_clair = predictions.rename(columns={
    'prediction_label': 'Prédiction (1=Retard)', 
    'prediction_score': 'Certitude (%)'
})

# On affiche juste les infos importantes
cols_a_afficher = ['Distance_km', 'Type_Transport', 'Prédiction (1=Retard)', 'Certitude (%)']
print(resultat_clair[cols_a_afficher])

Analyse des nouveaux dossiers en cours...


--------------------------------------------------
RÉSULTATS DE LA PRÉDICTION :
--------------------------------------------------
   Distance_km Type_Transport  Prédiction (1=Retard)  Certitude (%)
0          800         Camion                      0         0.6813
1           15          Avion                      0         0.6298


Et voila le résultat ! 

Si vous voulez vous pouvez parfaitement remplacer les données de test par de vraies données à l'étape 1

à l'étape 3 également

Cela aura pour conséquence de vous sortir de vrais scores pour la certitude